# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [4]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

In [5]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [6]:
# Import the necessary CSVs to Pandas DataFrames

file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [18]:
# Define Panel Visualization Functions

def housing_units_per_year():
    housing_units_year = sfo_data["housing_units"].groupby('year').mean()
    
    housing_data_min = housing_units_year.min()
    housing_data_max = housing_units_year.max()
    housing_data_std = housing_units_year.std()

    fig_housing_units = plt.figure()
    housing_units_plot = housing_units_year.plot.bar()
    housing_units_plot.set_ylim(housing_data_min - housing_data_std, housing_data_max + housing_data_std)
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight = True)

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    avg_rent_sqr_foot = sfo_data[["sale_price_sqr_foot", "gross_rent"]].groupby([sfo_data.index]).mean()
    fig_average_gross_rent = plt.figure()
    avg_gross_rent_plot = avg_rent_sqr_foot["gross_rent"].plot.line(title="Average Gross Rent in San Francisco", color = "magenta")
    avg_gross_rent_plot.set_xlabel("Year")
    avg_gross_rent_plot.set_ylabel("Gross Rent")
    plt.close(fig_average_gross_rent)
    return pn.pane.Matplotlib(fig_avg_gross_rent, tight = True)


def average_sales_price():
    """Average Sales Price Per Year."""
    
    fig_sale_price_sqr_foot = plt.figure()
    rent_sqt_plot = avg_rent_sqr_foot["sale_price_sqr_foot"].plot.line(title="Average Sales Price per Square Foot in San Francisco", color = "blue")
    rent_sqt_plot.set_xlabel("Year")
    rent_sqt_plot.set_ylabel("Avg. Sale Price")
    plt.close(fig_sale_price_sqr_foot)
    return pn.pane.Matplotlib(fig_avg_sales_price, tight = True)




def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    file_path = Path("Data/sfo_neighborhoods_census_data.csv")
    grouped_neighborhood_data = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    grouped_sales = grouped_neighborhood_data.reset_index()
    grouped_sales.head(10)
    
    avg_price_by_neighborhood = grouped_sales.hvplot.line(x='year', y='sale_price_sqr_foot', groupby = 'neighborhood')
    return avg_price_by_neighborhood

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    top_10_expensive_neighborhoods = sfo_data.groupby("neighborhood").mean().sort_values(by='sale_price_sqr_foot', ascending=False).head(10)
    top_10_expensive_neighborhoods.reset_index(inplace=True)
    top_expensive_neighborhoods = expensive_plot = top_10_expensive_neighborhoods.hvplot.bar(
         x="neighborhood",
         y="sale_price_sqr_foot",
         colormap="viridis",
         title="Top 10 Expensive Neighborhoods in SFO",
    height=500,
    rot=90,
    xlabel="Neighborhood",
    ylabel="Avg. Sale Price per Square Foot"
)
    return top_expensive_neighborhoods


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    grouped_sales = grouped_neighborhood_data.reset_index()
    grouped_sales.head(10)
    expensive_neighborhoods_group_sales = grouped_sales.hvplot.line(x='year', y='sale_price_sqr_foot', groupby = 'neighborhood')+ grouped_sales.hvplot.line(x='year', y='gross_rent', groupby = 'neighborhood')
    return expensive_neighborhoods_group_sales
        
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    def parallel_coordinates():
        parallel_coordinates_plot = px.parallel_coordinates(
        top_neighborhood_sale_price,
        dimensions=['sale_price_sqr_foot','housing_units','gross_rent'],
        color='sale_price_sqr_foot',
        labels={
                'sale_price_sqr_foot': 'Sale Price per Sq.ft.',
                'housing_units': 'Housing Units',
                'gross_rent': 'Gross Rent',
    },
    )
    
    return parallel_coordinates_plot


def parallel_categories():
    parallel_categories_plot = px.parallel_categories(
    top_neighborhood_sale_price,
    dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'],
    color='sale_price_sqr_foot',
    labels={
            'neighborhood':'Neighborhood',
            'sale_price_sqr_foot': 'Sale Price per Sq.ft.',
            'housing_units': 'Housing Units',
            'gross_rent': 'Gross Rent',
    },
    )
    
    return parallel_categories_plot



def neighborhood_map():
    """Neighborhood Map."""

    neighborhood_mean = sfo_data.groupby('neighborhood').mean().reset_index()
    
    combined_neighborhoods = pd.concat([neighborhood_locations, neighborhood_mean],  
              axis='columns').drop(columns=['neighborhood'])
    
    map = px.scatter_mapbox(
        combined_neighborhoods,
        lat="Lat",
        lon="Lon",
        color="gross_rent",
        size="sale_price_sqr_foot",
        title = "Average Sale Price per Sq Foot and Gross Rent in SF",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=12,
        zoom=10
 )
    plotly_panel = pn.pane.Plotly(map)
    plotly_panel._updates = True
    return plotly_panel


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [17]:
# Create a Title and structure of the Dashboard
welcome_panel = pn.Column(
    neighborhood_map()
)

yearly_market_analysis = pn.Row(
    housing_units_per_year(), 
    average_gross_rent(), 
    average_sales_price()
)

neighborhood_analysis = pn.Column(
    'Top Ten Most Expensive Neighborhoods',
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_plot_analysis = pn.Column(
    '## Parallel Coordinates and Categories',
    parallel_coordinates(),
    parallel_categories()
)

# Create tabs
panel = pn.Tabs(
    ('Welcome', welcome_panel),
    ('Yearly Market Analysis', yearly_market_analysis),
    ('Neighborhood Analysis', neighborhood_analysis),
    ('Parallel Plot Analysis', parallel_plot_analysis)
)

NameError: name 'neighborhood_locations' is not defined

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
panel.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [28]:
# housing_units_per_year()

In [29]:
# average_gross_rent()

In [30]:
# average_sales_price()

In [31]:
# average_price_by_neighborhood()

In [32]:
# top_most_expensive_neighborhoods()

In [33]:
# most_expensive_neighborhoods_rent_sales()

In [34]:
# neighborhood_map().show()

In [35]:
# parallel_categories()

In [36]:
# parallel_coordinates()

In [37]:
# sunburst()